In [1]:
import os
import sys
sys.path.append('/home/opencv/src/models/research')
sys.path.append('/home/opencv/src/models/research/slim/')
if 'PYTHONPATH' not in os.environ:
    os.environ['PYTHONPATH'] = ""
os.environ['PYTHONPATH'] += ':/home/opencv/src/models/research:/home/opencv/src/models/research/slim/'

In [2]:
# For the fruit model included in the repo below we have 240 training images
# For faster training time, images should be resized to 300x300 and then annotated
# Images should contain the objects of interest at various scales, angles, lighting conditions, locations
# For acceptable results - mAP@0.5 of 0.9 the model was trained with batch size of 24
# and 5000 steps. this takes about 1h using 2 augmentations. 
# using 5 augmentations it takes about 2h 
num_steps = 5000  # A step means using a single batch of data. larger batch, less steps required
#Number of evaluation steps.
num_eval_steps = 50
#Batch size 24 is a setting that generally works well. can be changed higher or lower 
MODELS_CONFIG = {
        'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 24
    }
}
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colab's GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [3]:
%cd /home/opencv/src/models/research
!protoc object_detection/protos/*.proto --python_out=.

!python object_detection/builders/model_builder_test.py

/home/opencv/src/models/research


In [4]:
#%%capture
repo_dir_path = '/home/opencv/src/object_detection_demo_flow'
%cd {repo_dir_path}
print(os.getcwd())

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

# Set the paths
test_record_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/test.record'
train_record_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/train.record'
label_map_pbtxt_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/label_map.pbtxt'

/home/opencv/src/object_detection_demo_flow
/home/opencv/src/object_detection_demo_flow
Successfully converted xml to csv.
Generate `data/annotations/label_map.pbtxt`
Successfully converted xml to csv.


W0820 10:39:42.430768 140469581649728 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0820 10:39:42.456622 140469581649728 module_wrapper.py:139] From generate_tfrecord.py:53: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /home/opencv/src/object_detection_demo_flow/data/annotations/train.record


W0820 10:39:44.210041 140336242124608 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0820 10:39:44.218693 140336242124608 module_wrapper.py:139] From generate_tfrecord.py:53: The name tf.gfile.GFile is deprecated. Please use 

In [5]:
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/home/opencv/src/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/home/opencv/src/models/research/pretrained_model
total 135M
drwxr-xr-x  3 opencv opencv 4.0K  3月 30  2018 .
drwxrwxr-x 23 opencv opencv 4.0K  8月 20 10:41 ..
-rw-r--r--  1 opencv opencv   77  3月 30  2018 checkpoint
-rw-r--r--  1 opencv opencv  67M  3月 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 opencv opencv  65M  3月 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 opencv opencv  15K  3月 30  2018 model.ckpt.index
-rw-r--r--  1 opencv opencv 3.4M  3月 30  2018 model.ckpt.meta
-rw-r--r--  1 opencv opencv 4.2K  3月 30  2018 pipeline.config
drwxr-xr-x  3 opencv opencv 4.0K  3月 30  2018 saved_model


In [6]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/home/opencv/src/models/research/pretrained_model/model.ckpt'

# Configuring a Training Pipeline

In [7]:
import os
pipeline_fname = os.path.join('/home/opencv/src/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [8]:
import re
iou_threshold = 0.50
num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    # Set number of classes num_classes.
    s = re.sub('iou_threshold: [0-9].[0-9]+',
               'iou_threshold: {}'.format(iou_threshold), s)
    
    f.write(s)

# Train the model

In [9]:
# necessary
#*********************** change folder name adding timestamp TODO *************
model_dir = 'training/'

# remove previous training result
!rm -rf ~/.nv
# !rm -rf {model_dir}
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Optionally remove content in output model directory for a fresh start.
# !rm -rf {model_dir}
# os.makedirs(model_dir, exist_ok=True)
!python /home/opencv/src/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

W0820 10:46:21.416286 140352541824832 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 5000
I0820 10:46:21.416452 140352541824832 config_util.py:552] Maybe overwriting train_steps: 5000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0820 10:46:21.416536 140352541824832 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0820 10:46:21.416609 140352541824832 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0820 10:46:21.416680 140352541824832 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0820 10:46:21.416768 140352541824832 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_t

Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0820 10:46:37.233152 140352541824832 api.py:332] From /home/opencv/anaconda2/envs/ssd/lib/python3.6/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:1004: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W0820 10:46:40.220083 140352541824832 deprecation.py:323] From /home/opencv/src/models/research/object_detection/inputs.py:259: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
I0820 10:46:43.149038 140352541824832 estimator.py:1148] Calling model_fn.
Instructions for updating:
Please use `layer.__call__`

INFO:tensorflow:Running local_init_op.
I0820 10:47:02.374111 140352541824832 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0820 10:47:02.792927 140352541824832 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into training/model.ckpt.
I0820 10:47:11.669468 140352541824832 basic_session_run_hooks.py:606] Saving checkpoints for 0 into training/model.ckpt.
2020-08-20 10:47:26.574233: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-08-20 10:47:27.817220: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
INFO:tensorflow:loss = 10.436161, step = 0
I0820 10:47:29.613238 140352541824832 basic_session_run_hooks.py:262] loss = 10.436161, step = 0
INFO:tensorflow:global_step/sec: 1.69173
I0820 10:48:28.723703 140352541824832 basic_session_run_hooks.py:692] global_step/sec

INFO:tensorflow:Running local_init_op.
I0820 10:57:19.853659 140352541824832 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0820 10:57:19.972480 140352541824832 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Performing evaluation on 10 images.
I0820 10:57:23.111056 140346830870272 coco_evaluation.py:237] Performing evaluation on 10 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0820 10:57:23.145033 140346830870272 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
I0820 10:57:23.147743 140346830870272 coco_tools.py:138] DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=  

INFO:tensorflow:depth of additional conv before box predictor: 0
I0820 11:07:17.882259 140352541824832 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:Done calling model_fn.
I0820 11:07:18.945087 140352541824832 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-20T11:07:18Z
I0820 11:07:18.957028 140352541824832 evaluation.py:255] Starting evaluation at 2020-08-20T11:07:18Z
INFO:tensorflow:Graph was finalized.
I0820 11:07:19.249344 140352541824832 monitored_session.py:240] Graph was finalized.
2020-08-20 11:07:19.249727: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-20 11:07:19.250109: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2060 SUPER major: 7 minor: 5 memoryClockRate(GHz): 1.6

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2159: training/model.ckpt-2159
I0820 11:07:23.524805 140352541824832 estimator.py:2109] Saving 'checkpoint_path' summary for global step 2159: training/model.ckpt-2159
INFO:tensorflow:global_step/sec: 1.55786
I0820 11:07:46.581526 140352541824832 basic_session_run_hooks.py:692] global_step/sec: 1.55786
INFO:tensorflow:loss = 1.9533004, step = 2200 (64.191 sec)
I0820 11:07:46.582180 140352541824832 basic_session_run_hooks.py:260] loss = 1.9533004, step = 2200 (64.191 sec)
INFO:tensorflow:global_step/sec: 1.79756
I0820 11:08:42.212618 140352541824832 basic_session_run_hooks.py:692] global_step/sec: 1.79756
INFO:tensorflow:loss = 1.6462892, step = 2300 (55.631 sec)
I0820 11:08:42.213192 140352541824832 basic_session_run_hooks.py:260] loss = 1.6462892, step = 2300 (55.631 sec)
INFO:tensorflow:global_step/sec: 1.80825
I0820 11:09:37.514810 140352541824832 basic_session_run_hooks.py:692] global_step/sec: 1.80825
INFO:tensorflow

INFO:tensorflow:Running local_init_op.
I0820 11:17:20.637190 140352541824832 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0820 11:17:20.757282 140352541824832 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Performing evaluation on 10 images.
I0820 11:17:23.577368 140346822477568 coco_evaluation.py:237] Performing evaluation on 10 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0820 11:17:23.577978 140346822477568 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
I0820 11:17:23.580485 140346822477568 coco_tools.py:138] DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=  

In [10]:
!pwd

/home/opencv/src/object_detection_demo_flow
